# 🛰️ Stiahnutie a spracovanie Sentinel-2 dát pre Slovensko (2024)

Tento notebook demonštruje, ako spracovať terabyte satelitných dáta zo systému Sentinel-2 (konkrétne Level-2A – povrchové odrazy), bez nutnosti ich sťahovania
- vyhľadáme satelitné snímky Sentinel-2 cez Planetary Computer,
- odstránime oblačnosť a tiene pomocou masky `SCL`,
- ponecháme len RGB + NIR + SWIR pásma,
- prevzorkujeme všetko na jednotné rozlíšenie 10 m.

🎯 Územie: Slovensko  
🗓️ Obdobie: rok 2024

In [2]:
# 🔧 Najskôr nainštalujeme potrebné knižnice
# Spustite len raz v Jupyter notebooku
!pip install pystac-client stackstac planetary-computer geopandas rioxarray --quiet

In [1]:
# 📚 Importujeme knižnice
import stackstac
import pystac_client
import xarray as xr
import numpy as np
from shapely.geometry import box
import datetime
import planetary_computer

## 🗺️ Definovanie územia a časového rozsahu

Zameriame sa na celé územie Slovenska a vyberieme dáta zo Sentinel-2 za celý rok 2024.

In [2]:
# 🗓️ Nastavíme dátumový rozsah – celý rok 2024
start_date = "2024-01-01"
end_date = "2024-12-31"

# 📦 Nastavíme hranice Slovenska (v stupňoch – WGS84)
bbox = [16.83, 47.73, 22.56, 49.60]  # [západ, juh, východ, sever]

## 🔍 Vyhľadanie Sentinel-2 dát cez STAC API

Použijeme tzv. STAC katalóg – štandardizovaný spôsob, ako vyhľadávať a sťahovať satelitné dáta z cloudu.
Dáta sú hostované na AWS (Amazon Web Services).

In [3]:
# 🌍 Pripojíme sa ku katalógu Planetary Computer
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

# Vyhľadáme Sentinel-2 Level-2A snímky nad Slovenskom
search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox,
    datetime=f"{start_date}/{end_date}",
    query={"eo:cloud_cover": {"lt": 80}},  # oblačnosť menej než 80 %
    max_items=500  # obmedzenie na rýchlu ukážku
)

# Autorizujeme každú položku pre použitie v Planetary Computer
items = [planetary_computer.sign(item) for item in search.items()]
print(f"Počet nájdených snímok: {len(items)}")

Počet nájdených snímok: 500


## 🎨 Výber pásiem a načítanie pomocou stackstac

Vyberieme len RGB (B02–B04), NIR (B08), SWIR (B11, B12) a SCL (klasifikácia scény pre maskovanie).

In [4]:
# Vybrané spektrálne pásma + SCL (na maskovanie)
bands = ["B02", "B03", "B04", "B08", "B11", "B12", "SCL"]

# Načítanie stacku
stack = stackstac.stack(
    items,
    assets=bands,
    resolution=10,  # všetky pásma (aj 20m) prevzorkujeme na 10m
    chunksize=1024,
    bounds_latlon=bbox,
    epsg = 32634
)
stack

<xarray.DataArray 'stackstac-f6e2a399efcd05dabef0563791add39c' (time: 500,
                                                                band: 7,
                                                                y: 21505,
                                                                x: 42964)> Size: 26TB
dask.array<fetch_raster_window, shape=(500, 7, 21505, 42964), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/42)
  * time                                     (time) datetime64[ns] 4kB 2024-0...
    id                                       (time) <U54 108kB 'S2B_MSIL2A_20...
  * band                                     (band) <U3 84B 'B02' ... 'SCL'
  * x                                        (x) float64 344kB 1.873e+05 ... ...
  * y                                        (y) float64 172kB 5.503e+06 ... ...
    s2:mean_solar_zenith                     (time) float64 4kB 48.87 ... 74.28
    ...                                       ...
    gsd                                      (band) float64 56B 10.0 ... 20.0
    title                                    (band) <U29 812B 'Band 2 - Blue ...
    common_name                              (band) object 56B 'blue' ... None
    center_wavelength                        (band) object 56B 0.49 ... None
    full_width_half_max                      (band) object 56B 0.098 ... None
    epsg                                     int64 8B 32634
Attributes:
    spec:        RasterSpec(epsg=32634, bounds=(187340, 5287470, 616980, 5502...
    crs:         epsg:32634
    transform:   | 10.00, 0.00, 187340.00|\n| 0.00,-10.00, 5502520.00|\n| 0.0...
    resolution:  10

## ☁️ Odstránenie oblakov, tieňov a iných artefaktov pomocou masky `SCL`

In [6]:
# Funkcia na maskovanie oblakov a tieňov
def mask_clouds(ds):
    scl = ds.sel(band="SCL")
    mask = xr.where(scl.isin([3, 8, 9, 10, 11]), 0, 1)  # tiene, oblačnosť, cirrus, saturácia
    return ds.drop_sel(band="SCL") * mask

# Aplikujeme masku
masked = mask_clouds(stack)

# Zmeníme typ dát a nahradíme nulu ako NaN
masked = masked.where(masked != 0).astype("float32") * 1e-4
masked

<xarray.DataArray 'stackstac-f6e2a399efcd05dabef0563791add39c' (time: 500,
                                                                band: 6,
                                                                y: 21505,
                                                                x: 42964)> Size: 11TB
dask.array<mul, shape=(500, 6, 21505, 42964), dtype=float32, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/37)
  * time                                     (time) datetime64[ns] 4kB 2024-0...
    id                                       (time) <U54 108kB 'S2B_MSIL2A_20...
  * band                                     (band) <U3 72B 'B02' ... 'B12'
  * x                                        (x) float64 344kB 1.873e+05 ... ...
  * y                                        (y) float64 172kB 5.503e+06 ... ...
    s2:mean_solar_zenith                     (time) float64 4kB 48.87 ... 74.28
    ...                                       ...
    eo:cloud_cover                           (time) float64 4kB 22.14 ... 62.65
    sat:orbit_state                          <U10 40B 'descending'
    instruments                              <U3 12B 'msi'
    s2:granule_id                            (time) <U62 124kB 'S2B_OPER_MSI_...
    s2:reflectance_conversion_factor         (time) float64 4kB 0.9902 ... 1.034
    epsg                                     int64 8B 32634

## 📊 Výpočet priemerného obrazu pre mesian september

Ako príklad vypočítame priemer všetkých snímok pre každé pásmo.

In [17]:
# 📅 Vyfiltrujeme len snímky zo september 2024
sep_masked = masked.sel(time=masked.time.dt.month == 9)

# ❗ Kontrola počtu dostupných snímok
print(f"Počet snímok v septemberi: {sep_masked.time.size}")

# 📊 Vypočítame priemer len za september
sep_mean = sep_masked.mean(dim="time", skipna=True)

# 🔍 Zobrazíme informácie o výsledku
sep_mean

Počet snímok v septemberi: 59


<xarray.DataArray 'stackstac-f6e2a399efcd05dabef0563791add39c' (band: 6,
                                                                y: 21505,
                                                                x: 42964)> Size: 22GB
dask.array<mean_agg-aggregate, shape=(6, 21505, 42964), dtype=float32, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * band                                     (band) <U3 72B 'B02' ... 'B12'
  * x                                        (x) float64 344kB 1.873e+05 ... ...
  * y                                        (y) float64 172kB 5.503e+06 ... ...
    s2:processing_baseline                   <U5 20B '05.11'
    constellation                            <U10 40B 'Sentinel 2'
    s2:product_type                          <U7 28B 'S2MSI2A'
    s2:saturated_defective_pixel_percentage  float64 8B 0.0
    s2:datatake_type                         <U8 32B 'INS-NOBS'
    sat:orbit_state                          <U10 40B 'descending'
    instruments                              <U3 12B 'msi'
    epsg                                     int64 8B 32634

## ✅ Záver

V tomto notebooku sme:
- použili Planetary Computer na vyhľadanie a prístup k Sentinel-2 dátam,
- aplikovali masku na odstránenie oblakov a tieňov,
- vybrali relevantné pásma a všetko prevzorkovali na 10m,
- vypočítali priemerný obraz za rok 2024.

🎯 Tento prístup má demonštrovať ako budeme v ďaľších krokoch pracovať zo satelitnými snímkami bez nutnosti ich sťahovania a ukladania.